# Exercise 6 | Support Vector Machines

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from scipy.io import loadmat
from scipy.optimize import minimize
from sklearn import preprocessing
from sklearn import svm

*
The code below tries best to replicate the coding paradigm from Andrew Ng's course. Some of the coding paradiam is not the best practice for scikit-learn library.
*

*
E.g.: When using SVM, instead of defining your own gaussian kernel, the default 'rbf' kernel defined by scikit-learn library is a better choice.
*

*
To evaluate the performance, the `SVC.score` function is better than computing the `mean` by hand.
*

## Part 1: Loading and Visualizing Data

We start the exercise by first loading and visualizing the
dataset. The following code will load the dataset into your
environment and plot the data.

In [ ]:
def plot_data(X, y):
    plt.figure(figsize=(10, 6))
    pos = np.where(y == 1)
    neg = np.where(y == 0)
    plt.plot(X[pos, 0], X[pos, 1], 'k+', linewidth=1, markersize=7);
    plt.plot(X[neg, 0], X[neg, 1], 'ko', markerfacecolor='y', markersize=7);

In [ ]:
mat = loadmat('ex6data1.mat')
X = mat['X']
y = mat['y']

plot_data(X, y)

# Part 2: Training Linear SVM

The following code will train a linear SVM on the dataset and plot the
decision boundary learned.

In [ ]:
def visualize_boundary_linear(X, y, model):
    w = model.coef_.flatten()
    b = model.intercept_
    X_min = X.min(axis=0)
    X_max = X.max(axis=0)
    xp = np.linspace(X_min[0], X_max[0], 100)
    yp = - (w[0] * xp + b) / w[1]
    plot_data(X, y)
    plt.plot(xp, yp, '-b');

In [ ]:
# You should try to change the C value below and see how the decision
# boundary varies (e.g., try C = 1000)
classifier = svm.LinearSVC(C=1)
model = classifier.fit(X, y.flatten())
visualize_boundary_linear(X, y, model)

## Part 3: Implementing Gaussian Kernel

In [ ]:
def gaussian_kernel(x1, x2, sigma):
    """returns a radial basis function kernel between x1 and x2"""
    
    x1 = x1.flatten()
    x2 = x2.flatten()
    
    return np.exp(- np.power(x1 - x2, 2).sum() / (2.0 * sigma ** 2))

In [ ]:
x1 = np.array([1, 2, 1])
x2 = np.array([0, 4, -1])
sigma = 2
sim = gaussian_kernel(x1, x2, sigma)

print(f'''Gaussian Kernel between x1 = [1; 2; 1], x2 = [0; 4; -1], sigma = 2:
\t{sim}
(this value should be about 0.324652)''')

## Part 4: Visualizing Dataset 2

In [ ]:
mat = loadmat('ex6data2.mat')
X = mat['X']
y = mat['y']

plot_data(X, y)

## Part 5: Training SVM with RBF Kernel (Dataset 2)

After you have implemented the kernel, we can now use it to train the
SVM classifier.

In [ ]:
def visualize_boundary(X, y, model):
    plot_data(X, y)
    X_min = X.min(axis=0)
    X_max = X.max(axis=0)

    x1plot = np.linspace(X_min[0], X_max[0], 100)
    x2plot = np.linspace(X_min[1], X_max[1], 100)
    X1, X2 = np.meshgrid(x1plot, x2plot)
    vals = model.predict(np.c_[X1.flat, X2.flat]).reshape(X1.shape)

    plt.contour(X1, X2, vals, [0], linecolor='blue', linewidth=3)

In [ ]:
%%time

C = 1
sigma = 0.05
gamma = np.power(sigma, -2)

def gaussian_matrix(X1, X2, sigma):
    m = np.zeros((X1.shape[0], X2.shape[0]))
    for i in range(X1.shape[0]):
        for j in range(X2.shape[0]):
            m[i, j] = gaussian_kernel(X1[i], X2[j], sigma)

    return m

model = svm.SVC(C=C, kernel=lambda X1, X2: gaussian_matrix(X1, X2, sigma)).fit(X, y.flatten())

In [ ]:
%%time

visualize_boundary(X, y, model)

## Part 6: Visualizing Dataset 3

In [ ]:
mat = loadmat('ex6data3.mat')
X = mat['X']
y = mat['y']
Xval = mat['Xval']
yval = mat['yval']

plot_data(X, y)

## Part 7: Training SVM with RBF Kernel (Dataset 3)

This a different dataset that you can use to experiment with. Try
different values of C and sigma here.

In [ ]:
c_options = [0.01, 0.1, 1, 10, 0.03, 0.3, 3, 30, 300]
# sigma_options = c_options
min_mean = 10 ** 10  # very large value

for c, s in ((c, s) for c in c_options
             for s in c_options):
    model = svm.SVC(C=c, kernel=lambda X1, X2: gaussian_matrix(X1, X2, s)).fit(X, y.ravel())
    predictions = model.predict(Xval)
    temp_mean = np.mean(predictions != yval.ravel())
    if temp_mean < min_mean:
        C = c
        sigma = s
        min_mean = temp_mean

print(f'C = {C}, sigma = {sigma}, mean = {min_mean}')
model = svm.SVC(C=C, kernel=lambda X1, X2: gaussian_matrix(X1, X2, sigma)).fit(X, y.ravel())
visualize_boundary(X, y, model)